In [ ]:
ad = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#TENANT_ID#')
sp = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#SP_ID#')
se = dbutils.secrets.get('#SERVICE_PRINCIPAL_NAME#', '#SECRET_KEY#')

 
spark.conf.set("fs.azure.account.auth.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", sp)
spark.conf.set("fs.azure.account.oauth2.client.secret.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", se)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "https://login.microsoftonline.com/" + ad + "/oauth2/token")

In [ ]:
import dlt
from pyspark.sql.functions import struct
from pyspark.sql.functions import col
from pyspark.sql.types import DateType
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
import re
import itertools
import numpy as np
from delta.tables import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType, DecimalType
from pyspark.sql.functions import regexp_replace

In [ ]:
# BedOccupancyForecast_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/HealthCarePredctiveAnalytics/BedOccupancyForecasted.csv"
# from pyspark.sql.functions import col

# @dlt.table()
# def BedOccupancyForecast_bronze():
#   return (spark.read.format("csv").option("header",True).load(BedOccupancyForecast_file))

# @dlt.table()
# def BedOccupancyForecast_silver():
#   return (dlt.read("BedOccupancyForecast_bronze"))

# @dlt.table()
# def BedOccupancyForecast_gold():
#   return (dlt.read("BedOccupancyForecast_silver"))

In [ ]:
%sql

select * from csv.`abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/OccupancyHealthcareData.csv`

In [ ]:

ReadmissionPrediction_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/HealthCarePredctiveAnalytics/ReadmissionPrediction.csv"

@dlt.table()
def ReadmissionPrediction_input_bronze():
  return (spark.read.format("csv").option("header",True).load(ReadmissionPrediction_input_file))

@dlt.table()
def ReadmissionPrediction_input_silver():
  return (dlt.read("ReadmissionPrediction_input_bronze").select(col("City").alias("City"),col("Disease").alias("Disease"),col("ReadmissionFactors").alias("ReadmissionFactors"),col("Month").alias("MonthName"),col("Actual_Readmission_Rate").alias("ActualReadmissionRate"),col("Predicted_Readmission_Rate").alias("PredictedReadmissionRate"),col("Flag").alias("Flag"),col("length_of_stay").alias("LengthOfStay"),col("Year").alias("Year"),col("Predicted_Readmission_Rate2").alias("PredictedReadmissionRateSecond")))

@dlt.table()
def ReadmissionPrediction_input_gold():
  return (dlt.read("ReadmissionPrediction_input_silver"))


In [ ]:

PatientWaitTimeData = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/HealthCarePredctiveAnalytics/WaitTimeForecast.csv"
@dlt.table()
def PatientWaitTime_bronze():
  return (spark.read.format("csv").option("header",True).load(PatientWaitTimeData))

@dlt.table()
def PatientWaitTime_silver():
  return (dlt.read("PatientWaitTime_bronze").select(col("date").alias("Date"),col("wait_time").alias("WaitTime"),col("Month").alias("MonthNumber"),col("Flag").alias("Flag"),col("Year").alias("Year")))

@dlt.table()
def PatientWaitTime_gold():
  return (dlt.read("PatientWaitTime_silver"))

In [ ]:
PredAnomalyData = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/HealthCarePredctiveAnalytics/PredAnomaly.csv"
@dlt.table()
def PredAnomalyData_bronze():
  return (spark.read.format("csv").option("header",True).load(PredAnomalyData))

@dlt.table()
def PredAnomalyData_silver():
  return (dlt.read("PredAnomalyData_bronze").select(col("PrincipalComponent1").alias("PrincipalComponentFirst"),col("PrincipalComponent2").alias("PrincipalComponentSecond"),col("PrincipalComponent3").alias("PrincipalComponentThird"),col("PatientID").alias("PatientID"),col("url").alias("URL"),col("AnomalyDetected").alias("AnomalyDetected")))

@dlt.table()
def PredAnomalyData_gold():
  return (dlt.read("PredAnomalyData_silver"))